#Parser
First of all what does parsing data mean?
Parsing data refers to the process of analyzing a sequence of symbols, often in the form of text or code, to understand its structure and extract relevant information.

what does a parser do?  A parser is a program or a module within a program that performs the task of parsing.


## langchain output parser
Language models generate text output, but for more structured information, output parsers are employed. Output parsers are classes with two essential methods.

1. Get format instructions: This method returns a string containing guidelines on how the language model's output should be formatted.

2. Parse : This method takes a string (assumed to be the language model's response) as input and processes it, transforming the unstructured text into a more organized and structured format.

Basically, the output parser will provide the main information we want in a suitable data structure.

*  [Output parsers langchain website](https://python.langchain.com/docs/modules/model_io/output_parsers/)
* [Output parsers langchain api](https://api.python.langchain.com/en/latest/api_reference.html?highlight=output_parsers#module-langchain.output_parsers)


Depending on our task, we have the flexibility to work with different parsers. Specifically, we will be utilizing the list comma-separated output parser and the Pydantic output parser.
1.
  * [sourc code of list parser](https://api.python.langchain.com/en/v0.0.342/_modules/langchain_core/output_parsers/list.html#)
  * [comma-separated  parser tutorial ](https://python.langchain.com/docs/modules/model_io/output_parsers/comma_separated)
  * [comma-separated output parser api ](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html?highlight=comma#)

2.
  * [pydantic website](https://docs.pydantic.dev/latest/)
  * [langchain Pydantic parser website](https://python.langchain.com/docs/modules/model_io/output_parsers/pydantic)
  * [langchain Pydantic parser api](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.pydantic.PydanticOutputParser.html?highlight=pydanticoutputparser#langchain.output_parsers.pydantic.PydanticOutputParser)

## Initialization section

In [1]:
!pip install langchain
!pip install openai==v0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## Import section

In [23]:
import os

from typing import List


from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.list import CommaSeparatedListOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser



from langchain.pydantic_v1 import BaseModel, Field

## preparation

In [3]:
openai_api_key = ...
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
llm = ChatOpenAI(temperature=0)

## comma-separated
We are creating a program that takes the name of a food and returns the list of ingredients needed to cook it

In [7]:
output_parser = CommaSeparatedListOutputParser()
output_parser_format =  output_parser.get_format_instructions()

In [8]:
promt_food_ingredients = """
You are a helpful AI that will provide a list of ingredients needed to cook a dish.

format of output: {format_output}
"""

In [14]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", promt_food_ingredients),
        ("user", "{food_name}"),
    ],
)
chat_template = chat_template.partial(format_output=output_parser_format)

In [19]:
food_name = input("Which specific dish would you like to know the ingredients for?")
message = chat_template.format_messages(food_name=food_name)
response = llm(message)
output = output_parser.parse(response.content)
print(f"""
type:{type(output)}


contain:{output}
""")

Which specific dish would you like to know the ingredients for?pizza

type:<class 'list'>


contain:['dough', 'tomato sauce', 'cheese', 'toppings (e.g. pepperoni', 'mushrooms', 'onions', 'etc.)']



## Pydantic
Now we are going to get more information

In [24]:
class DishRecipe(BaseModel):
    food_name: str = Field(description="Name of the food")
    ingredients: List[str] = Field(description="List of ingredients needed for cooking")
    cooking_steps: List[str] = Field(description="List of steps needed to cook the food")
parser = PydanticOutputParser(pydantic_object=DishRecipe)
format_output = parser.get_format_instructions()

In [42]:
promt_recipe = """
You are a helpful AI that users can ask for a food name, and you will provide the list of ingredients needed along with the steps required to cook it.
{format_output}
"""

In [43]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", promt_recipe),
        ("user", "{food_name}"),
    ],
)
chat_template = chat_template.partial(format_output=format_output)

In [46]:
food_name = input("Which specific dish would you like to know how to cook?")
message = chat_template.format_messages(food_name=food_name)
response = llm(message)
output = parser.parse(response.content)
print(f"""
contain:{output}
""")

Which specific dish would you like to know how to cook?pizza

contain:food_name='Pizza' ingredients=['Pizza dough', 'Tomato sauce', 'Mozzarella cheese', 'Toppings of your choice (e.g. pepperoni, mushrooms, onions, bell peppers)', 'Olive oil', 'Salt', 'Dried oregano', 'Garlic powder'] cooking_steps=['Preheat the oven to 475°F (245°C).', 'Roll out the pizza dough on a floured surface to your desired thickness.', 'Transfer the rolled-out dough to a baking sheet or pizza stone.', 'Spread a thin layer of tomato sauce evenly over the dough, leaving a small border around the edges.', 'Sprinkle mozzarella cheese over the sauce.', 'Add your desired toppings, such as pepperoni, mushrooms, onions, and bell peppers.', 'Drizzle a little olive oil over the toppings and sprinkle with salt, dried oregano, and garlic powder.', 'Bake the pizza in the preheated oven for about 12-15 minutes, or until the crust is golden brown and the cheese is melted and bubbly.', 'Remove the pizza from the oven and let i

## Task
write a program that takes the name of a plant as input and returns a summary of information, including average life expectancy and a list of locations where it can live and so on .